In [4]:
import numpy as np
import torch
import pandas as pd
# import nltk
import os

In [5]:
import random

In [6]:
from preprocessing import clean_text

In [7]:
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_word = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\muz94\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
train_data = pd.read_csv("train/train.csv", dtype={'id': int, 'qid1':int, 'qid2': int, 'question1':str, 'question2':str, 'is_duplicate': int})
train_data.set_index('id', inplace = True)

In [9]:
data = train_data.sample(n=100000).copy()
data.loc[data['is_duplicate']==1]

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
199481,300983,300984,How much initial capital investment would I re...,How much initial capital investment would I re...,1
190458,289543,289544,What exactly is the Arab Spring?,What is Arab Spring?,1
286653,407280,407281,Why should people switch to LED bulbs?,What should I pay attention to when buying LED...,1
40544,60811,11501,How do I tell my religious family I don't beli...,How do I tell my parent I don't believe in God?,1
46093,82485,82486,Is using essay writing services ethical?,Are essay writing services ethical?,1
...,...,...,...,...,...
38603,70083,70084,How does RSA decryption work?,Can you answer my RSA question?,1
51363,5988,12951,What is the expected KVPY 2016 SA cut off?,What do you think about the KVPY 2016 paper?,1
230732,143875,131004,"Would Modi ""dare"" to abolish the reservation?",Will PM Modi abolish caste based reservations?,1


In [10]:
def sen2list(sentence: str):
    mod_lst = []
    try:
        sentence = clean_text(sentence)

        lst = word_tokenize(sentence)
        
        #ps = PorterStemmer()
        for word in lst:
            c = word[0]
            if word != "" and (("a"<=c and c <="z") or ("A" <= c and c <="Z") or ("0"<= c and c <="9")):
                #mod_word = ps.stem(word)
                mod_word = word
                if mod_word not in stop_word:
                    mod_lst.append(mod_word)
        
    except:
        print(sentence)
    
    return mod_lst

In [11]:
data['lst1'] = data.apply(lambda row: sen2list(row.question1), axis = 1)
data['lst2'] = data.apply(lambda row: sen2list(row.question2), axis = 1)

nan


In [14]:
data

,qid1,qid2,question1,question2,is_duplicate,lst1,lst2
id,,,,,,,
191740,291166,291167,Do we really need an election commission in In...,Is election commission of India poorly framed?,0,"[really, need, election, commission, India]","[election, commission, India, poorly, framed]"
89303,150062,150063,"What is the right way to say ""you're welcome"" ...","What is the right way to say ""live free"" in It...",0,"[right, way, say, welcome, italian]","[right, way, say, live, free, italian]"
124380,200861,200862,How could Russia hack US election?,Can the US redo the 2016 Presidential election...,0,"[could, russia, hack, america, election]","[america, redo, 2016, presidential, election, ..."
199481,300983,300984,How much initial capital investment would I re...,How much initial capital investment would I re...,1,"[much, initial, capital, investment, would, re...","[much, initial, capital, investment, would, re..."
190458,289543,289544,What exactly is the Arab Spring?,What is Arab Spring?,1,"[exactly, arab, spring]","[arab, spring]"
...,...,...,...,...,...,...,...
328666,455240,455241,What is the most touching real life love story...,What are some examples of patents simple enoug...,0,"[touching, real, life, love, story, come, across]","[examples, patents, simple, enough, beginner, ..."
384803,45489,125363,Which is the best laptop for gaming under 60k ...,Which is best gaming laptop under 60000?,1,"[best, laptop, gaming, 60000, inr]","[best, gaming, laptop, 60000]"
134515,214973,214974,What are potato bugs and are they harmful to h...,How harmful could it be to have bugs in the re...,0,"[potato, bugs, harmful, humans]","[harmful, could, bugs, refrigerator]"


In [13]:
data = data.loc[(data['lst1'].str.len() > 0) & (data['lst2'].str.len() > 0),:].copy()

### Corpus

In [15]:
from gensim import corpora

In [16]:
corpus = data['lst1'].tolist() + data['lst2'].tolist()
len(corpus)

199916

In [17]:
dictionary = corpora.Dictionary(corpus)
len(dictionary.token2id)

45864

In [18]:
max_seq_len = 50
def text_to_seq(lst: list) -> list:
    if len(lst) <= max_seq_len:
        seq =  [dictionary.token2id[word]+1 for word in lst]
    else:
        new_lst = list(set(lst))
        if len(new_lst) <= max_seq_len:
            seq = [dictionary.token2id[word]+1 for word in new_lst]
        else:
            seq = [dictionary.token2id[new_lst[i]]+1 for i in sorted(random.sample(range(len(new_lst)), max_seq_len))]
    return seq

In [19]:
data['seq1'] = data.apply(lambda row: text_to_seq(row.lst1), axis = 1)
data['seq2'] = data.apply(lambda row: text_to_seq(row.lst2), axis = 1)

In [20]:
data

,qid1,qid2,question1,question2,is_duplicate,lst1,lst2,seq1,seq2
id,,,,,,,,,
191740,291166,291167,Do we really need an election commission in In...,Is election commission of India poorly framed?,0,"[really, need, election, commission, India]","[election, commission, India, poorly, framed]","[5, 4, 3, 2, 1]","[3, 2, 1, 9829, 26338]"
89303,150062,150063,"What is the right way to say ""you're welcome"" ...","What is the right way to say ""live free"" in It...",0,"[right, way, say, welcome, italian]","[right, way, say, live, free, italian]","[7, 9, 8, 10, 6]","[7, 9, 8, 262, 528, 6]"
124380,200861,200862,How could Russia hack US election?,Can the US redo the 2016 Presidential election...,0,"[could, russia, hack, america, election]","[america, redo, 2016, presidential, election, ...","[12, 14, 13, 11, 3]","[11, 7538, 737, 3414, 3, 10696, 14, 4225, 622]"
199481,300983,300984,How much initial capital investment would I re...,How much initial capital investment would I re...,1,"[much, initial, capital, investment, would, re...","[much, initial, capital, investment, would, re...","[20, 18, 17, 19, 25, 21, 23, 22, 15, 16, 24]","[20, 18, 17, 19, 25, 21, 23, 15, 16, 24]"
190458,289543,289544,What exactly is the Arab Spring?,What is Arab Spring?,1,"[exactly, arab, spring]","[arab, spring]","[27, 26, 28]","[26, 28]"
...,...,...,...,...,...,...,...,...,...
328666,455240,455241,What is the most touching real life love story...,What are some examples of patents simple enoug...,0,"[touching, real, life, love, story, come, across]","[examples, patents, simple, enough, beginner, ...","[539, 121, 119, 1221, 898, 572, 3560]","[117, 15871, 1311, 1851, 2140, 1266]"
384803,45489,125363,Which is the best laptop for gaming under 60k ...,Which is best gaming laptop under 60000?,1,"[best, laptop, gaming, 60000, inr]","[best, gaming, laptop, 60000]","[172, 2112, 4945, 7230, 467]","[172, 4945, 2112, 7230]"
134515,214973,214974,What are potato bugs and are they harmful to h...,How harmful could it be to have bugs in the re...,0,"[potato, bugs, harmful, humans]","[harmful, could, bugs, refrigerator]","[12249, 9165, 1292, 1253]","[1292, 12, 9165, 7467]"


### Split

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(data[["seq1", "seq2"]], data["is_duplicate"], test_size=0.2, random_state=42, stratify=data["is_duplicate"].values)

In [28]:
X_train

,seq1,seq2
id,,
153275,"[108, 910, 1413]","[108, 910]"
114523,"[2683, 2171, 9157]","[186, 18050, 522]"
262353,"[3876, 6143, 670, 671]","[3876, 201]"
154306,"[172, 917, 6747, 3939, 4304]","[172, 917, 6747, 3939, 4304]"
167944,"[1182, 208, 1937, 2451, 35037, 458, 3941, 3503...","[45205, 208, 2058, 20]"
...,...,...
338657,"[80, 1773, 3246, 1382]","[3246, 152, 80]"
332153,"[1422, 1797, 6284, 2313, 13980]","[25, 2704, 8866, 15231, 2313, 13980, 1422, 267..."
132674,"[478, 208, 24060]","[1111, 1528, 1529, 311]"


In [29]:
y_train

id
153275    0
114523    0
262353    1
154306    1
167944    0
         ..
338657    0
332153    0
132674    0
357337    1
235002    1
Name: is_duplicate, Length: 79966, dtype: int32

# Import Glove

In [63]:
EMBEDDING_FILE = 'glove.6B.100d.txt'
embed_dict = {}
for o in open(EMBEDDING_FILE, encoding='utf-8'):
    o_lst = o.split(" ")
    embed_dict[o_lst[0]] = np.asarray(o_lst[1:], dtype='float32')[:100]
embed_dict['the'].shape

(100,)

In [64]:
len(embed_dict.keys())

400001

In [65]:
vocab_size = len(dictionary.token2id) + 1
substi_vector = np.asarray([0 for n in range(100)], dtype = 'float32')
      
# Adding again 1 because of reserved 0 index
subs_num = 1
embedding_matrix = np.zeros((vocab_size,100))
ps = PorterStemmer()
for word, idx in dictionary.token2id.items():
    if word in embed_dict.keys():
        embedding_matrix[idx+1] = embed_dict[word]
    else:
        mod_word = ps.stem(word)
        if mod_word in embed_dict.keys():
            embedding_matrix[idx+1] = embed_dict[mod_word]
        else:
            embedding_matrix[idx+1] = substi_vector
            subs_num += 1
        
embedding_matrix[0] = substi_vector

In [66]:
subs_num

7701

In [67]:
(embedding_matrix.shape, embedding_matrix.dtype)

((45865, 100), dtype('float64'))

# DataLoader

##### Test

In [59]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [77]:
dataloader1_test = [torch.tensor([n], dtype = torch.long) for n in range(10)]
dataloader2_test = [torch.tensor([n], dtype = torch.long) for n in range(10, 20)]
dataloader3_test = [torch.tensor([random.choice([0,1])], dtype = torch.long) for n in range(20, 30)]
(dataloader1_test, dataloader2_test, dataloader3_test)

([tensor([0]),
  tensor([1]),
  tensor([2]),
  tensor([3]),
  tensor([4]),
  tensor([5]),
  tensor([6]),
  tensor([7]),
  tensor([8]),
  tensor([9])],
 [tensor([10]),
  tensor([11]),
  tensor([12]),
  tensor([13]),
  tensor([14]),
  tensor([15]),
  tensor([16]),
  tensor([17]),
  tensor([18]),
  tensor([19])],
 [tensor([0]),
  tensor([1]),
  tensor([0]),
  tensor([1]),
  tensor([1]),
  tensor([1]),
  tensor([0]),
  tensor([1]),
  tensor([0]),
  tensor([0])])

In [78]:
dataloader_test_train = MyDataset(dataloader1_test, dataloader2_test, dataloader3_test)

In [79]:
dataloader_test_train.__len__()

10

In [80]:
dataloader_test_train.__getitem__(5)

(tensor([5]), tensor([15]), tensor([1]))

In [81]:
class MyDataset(torch.utils.data.Dataset):
    r"""Dataset wrapping tensors.

    Each sample will be retrieved by indexing tensors along the first dimension.

    Args:
        *tensors (Tensor): tensors that have the same size of the first dimension.
    """
    #tensors: Tuple[Tensor, ...]

    def __init__(self, lst1, lst2, lst3) -> None:
        assert len(lst1) == len(lst2) and len(lst1) == len(lst3), "Size mismatch between tensors"
        self.lst1 = lst1
        self.lst2 = lst2
        self.lst3 = lst3

    def __getitem__(self, index):
        return tuple([self.lst1[index], self.lst2[index], self.lst3[index]])

    def __len__(self):
        return len(self.lst1)

In [82]:
tuple([1,2,3])

(1, 2, 3)

In [83]:
test_train_dataloader = DataLoader(
            dataloader_test_train,  # The training samples.
            shuffle=True, # Select batches randomly
            batch_size = 4, # Trains with this batch size.
            num_workers = 0
        )

In [84]:
for batch in test_train_dataloader:
    print(batch)

[tensor([[6],
        [8],
        [0],
        [7]]), tensor([[16],
        [18],
        [10],
        [17]]), tensor([[0],
        [0],
        [0],
        [1]])]
[tensor([[2],
        [5],
        [4],
        [9]]), tensor([[12],
        [15],
        [14],
        [19]]), tensor([[0],
        [1],
        [1],
        [0]])]
[tensor([[1],
        [3]]), tensor([[11],
        [13]]), tensor([[1],
        [1]])]


#### New way

In [30]:
X_train_1 = [torch.tensor(np.asarray([lst], dtype='long'), dtype=torch.long) for lst in X_train['seq1'].values]
X_train_2 = [torch.tensor(np.asarray([lst], dtype='long'), dtype=torch.long) for lst in X_train['seq2'].values]

Y_train =  [torch.tensor(np.asarray([lst], dtype='float32'), dtype=torch.float32) for lst in y_train.values]

(len(X_train_1), len(X_train_2), len(Y_train))

(79966, 79966, 79966)

In [167]:
train_dataset = [tuple([X_train_1[n], X_train_2[n], Y_train[n]]) for n in range(len(X_train_1))]
len(train_dataset)

79968

In [168]:
train_dataset[0]

(tensor([[  282,   155,  4880, 14420]]),
 tensor([[4880, 6540, 5423,  282,  155, 4880]]),
 tensor([1.]))

In [31]:
from sklearn.utils import shuffle

In [32]:
train_dataset = shuffle(train_dataset)

NameError: name 'train_dataset' is not defined

In [127]:
len(train_dataset)

79968

In [128]:
train_dataset[:10]

[(tensor([  11,  318,  395, 4892]),
  tensor([  461, 19277,   395,  4892]),
  tensor(1.)),
 (tensor([  65,   97, 5904,  647, 3199]),
  tensor([ 647, 5291,   97, 5160]),
  tensor(1.)),
 (tensor([  384,   385,   591,   733,  5018, 18042,   145]),
  tensor([  384,   385,   591,   733,  5018, 18042]),
  tensor(0.)),
 (tensor([475,  97,  57]), tensor([205, 475,  97]), tensor(1.)),
 (tensor([10975,   318, 35486]),
  tensor([ 745, 1197,   42,   11,   26,  728, 6860]),
  tensor(0.)),
 (tensor([  68, 4126]), tensor([  68, 4126,   26]), tensor(0.)),
 (tensor([18151,  2931]), tensor([18151,  2931,  8242]), tensor(0.)),
 (tensor([154, 155, 172, 414, 750]),
  tensor([ 154,  155, 1008,   34,  172]),
  tensor(1.)),
 (tensor([2316,  686]), tensor([2316,  686]), tensor(1.)),
 (tensor([1366, 1362,  874,  547, 1075, 1363, 1364,  874,  547, 1365]),
  tensor([2532,  999, 6974,  874,  547, 1365, 1362]),
  tensor(0.))]

##### Validation

In [33]:
X_valid_1 = [torch.tensor(np.asarray([lst], dtype='long'), dtype=torch.long) for lst in X_validation['seq1'].values]
X_valid_2 = [torch.tensor(np.asarray([lst], dtype='long'), dtype=torch.long) for lst in X_validation['seq2'].values]

Y_valid =  [torch.tensor(np.asarray([lst], dtype='float32'), dtype=torch.float32) for lst in y_validation.values]

# BiLSTM

##### <span style="color:blue">Test</span>

In [54]:
batch_sample = train_dataset[0]

Embedding

In [59]:
test_embedding = nn.Embedding(embedding_dim, 100)
test_embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32)) 

In [60]:
embeds1 = test_embedding(batch_sample[0])
embeds2 = test_embedding(batch_sample[1])

In [63]:
(embeds1.shape, embeds2.shape)

(torch.Size([1, 4, 100]), torch.Size([1, 6, 100]))

In [62]:
embeds1 = embeds1.view(1,embeds1.size(0), embeds1.size(1))
embeds2 = embeds2.view(1,embeds2.size(0), embeds2.size(1))

Bilstm

In [73]:
test_lstm1 = nn.LSTM(input_size = 100, hidden_size = hidden_dim, batch_first=True, bidirectional = True)

In [74]:
lstm_out1, hc1 = test_lstm1(embeds1)

In [75]:
lstm_out2,_ = test_lstm1(embeds2)

In [76]:
(lstm_out1.shape, lstm_out2.shape)

(torch.Size([1, 4, 100]), torch.Size([1, 6, 100]))

In [77]:
lstm_out1

tensor([[[-0.0073,  0.1217, -0.0217,  0.1182, -0.0436,  0.0426,  0.0137,
           0.0694,  0.0949,  0.1114, -0.1540,  0.1062, -0.0846,  0.1304,
          -0.1763,  0.1521, -0.1003, -0.0505, -0.0207,  0.1399, -0.1157,
          -0.0611, -0.2948, -0.1102,  0.0740,  0.0203, -0.1087,  0.0282,
          -0.0162,  0.0891,  0.0488, -0.0193,  0.0528,  0.0944,  0.1920,
          -0.0274, -0.0781, -0.0602, -0.0824, -0.1415,  0.1435,  0.0725,
           0.1375,  0.0440,  0.0505, -0.0192,  0.1977,  0.1542, -0.0134,
           0.1737, -0.1448, -0.2157, -0.0889,  0.1663, -0.2169,  0.0549,
           0.2533,  0.0240, -0.1593, -0.0357, -0.0649,  0.0618,  0.0010,
           0.0644,  0.1356, -0.1475,  0.1889, -0.2118, -0.0790,  0.0768,
           0.2867,  0.0609,  0.2869,  0.0344, -0.1935,  0.1714,  0.0324,
           0.1643, -0.2180, -0.0319, -0.0578,  0.2311,  0.0850,  0.0341,
           0.0640, -0.2028, -0.0404, -0.0095,  0.1074, -0.3042, -0.0905,
          -0.0535, -0.1194, -0.1921, -0.0334, -0.07

In [78]:
hc1

(tensor([[[-0.0306,  0.1480, -0.2445,  0.0267,  0.0208, -0.1850, -0.0981,
            0.1398, -0.1411,  0.0620,  0.0233,  0.1925, -0.1007,  0.2788,
            0.0654, -0.0724, -0.0561, -0.1529,  0.0128, -0.0602,  0.2552,
            0.0079, -0.1903, -0.1445, -0.0123,  0.0041, -0.1440,  0.1949,
            0.0252,  0.0752, -0.1384,  0.1050,  0.2613,  0.2094,  0.0180,
           -0.0249, -0.0144, -0.2120, -0.1087,  0.0555, -0.0110,  0.0707,
            0.1783,  0.0598,  0.0378,  0.3479,  0.4340,  0.1660, -0.0746,
           -0.1366]],
 
         [[-0.1448, -0.2157, -0.0889,  0.1663, -0.2169,  0.0549,  0.2533,
            0.0240, -0.1593, -0.0357, -0.0649,  0.0618,  0.0010,  0.0644,
            0.1356, -0.1475,  0.1889, -0.2118, -0.0790,  0.0768,  0.2867,
            0.0609,  0.2869,  0.0344, -0.1935,  0.1714,  0.0324,  0.1643,
           -0.2180, -0.0319, -0.0578,  0.2311,  0.0850,  0.0341,  0.0640,
           -0.2028, -0.0404, -0.0095,  0.1074, -0.3042, -0.0905, -0.0535,
           -0.

Matching layer

In [72]:
test_matchpara1 = nn.Parameter(torch.rand(match_size, hidden_dim))
test_matchpara2 = nn.Parameter(torch.rand(match_size, hidden_dim))
test_cosSim = nn.CosineSimilarity(dim=1)

In [82]:
test_cat1 = torch.tensor([1,2,3])
test_cat2 = torch.tensor([3,4])
torch.cat((test_cat1, test_cat2), dim=0)

tensor([1, 2, 3, 3, 4])

In [84]:
def test_matching_method(batch1_t, lstm_out2_batch1):
    prep_batch1_t = batch1_t.repeat(lstm_out2_batch1.size(0),1)
    weight = test_cosSim(prep_batch1_t, lstm_out2_batch1).detach().numpy()
    max_weight = np.argmax(weight)
    return lstm_out2_batch1[max_weight,:]

def test_match_cal(lstm_out1, lstm_out2):
    match_out_all = []
    for b in range(lstm_out1.size(0)):
        match_out_lst = []
        lstm_out1_batch1 = lstm_out1[b,:,:]
        #lstm_out2_batch1_end = lstm_out2[b, -1, :]
        for seq in range(lstm_out1_batch1.size(0)):
            batch1_t_first = lstm_out1_batch1[seq,:hidden_dim]
            #batch2_t = lstm_out2_batch1_end
            batch2_t_first = test_matching_method(batch1_t_first, lstm_out2[b,:,:hidden_dim])
            mod_batch1_t_first = torch.mul(test_matchpara1, batch1_t_first)
            mod_batch2_t_first = torch.mul(test_matchpara1, batch2_t_first)
            match_out_t_first = test_cosSim(mod_batch1_t_first, mod_batch2_t_first)
            batch1_t_second = lstm_out1_batch1[seq,hidden_dim:]
            batch2_t_second = test_matching_method(batch1_t_second, lstm_out2[b,:,hidden_dim:])
            mod_batch1_t_second = torch.mul(test_matchpara2, batch1_t_second)
            mod_batch2_t_second = torch.mul(test_matchpara2, batch2_t_second)
            match_out_t_second = test_cosSim(mod_batch1_t_second, mod_batch2_t_second)
            match_out_t = torch.cat((match_out_t_first, match_out_t_second), dim=0)
            match_out_lst.append(match_out_t)
        match_out = torch.stack(match_out_lst)
        match_out_all.append(match_out)
    return torch.stack(match_out_all)

In [86]:
test_match1 = test_match_cal(lstm_out1, lstm_out2)
test_match2 = test_match_cal(lstm_out2, lstm_out1)

In [87]:
(test_match1.size(), test_match2.size())

(torch.Size([1, 4, 60]), torch.Size([1, 6, 60]))

New lstm

In [95]:
test_lstm_new1 = nn.LSTM(input_size = match_size*2, hidden_size = hidden_dim2, batch_first=True, bidirectional = True)

In [99]:
_,(test_lstm_new_out1,_) = test_lstm_new1(test_match1)
_,(test_lstm_new_out2,_) = test_lstm_new1(test_match2)

In [100]:
(test_lstm_new_out1.size(), test_lstm_new_out2.size())

(torch.Size([2, 1, 20]), torch.Size([2, 1, 20]))

Predict

In [101]:
test_lstm_new_out1 = test_lstm_new_out1.view(test_lstm_new_out1.size(0), test_lstm_new_out1.size(2))
test_lstm_new_out2 = test_lstm_new_out2.view(test_lstm_new_out2.size(0), test_lstm_new_out2.size(2))

In [102]:
test_lstm_new_out1

tensor([[-5.1465e-01, -1.6745e-01,  3.1715e-02,  1.8416e-01, -1.7783e-01,
         -2.7314e-01,  2.8250e-01,  4.5750e-02, -2.4371e-01, -2.4981e-02,
          3.5792e-02, -1.4749e-01, -3.1761e-01, -2.4772e-01,  4.9829e-01,
         -4.7374e-02, -5.3364e-04,  4.8119e-01,  3.1403e-02,  1.5268e-01],
        [ 1.4047e-02,  3.5429e-01, -3.0918e-02, -1.1251e-01, -3.2316e-01,
         -3.1126e-01, -1.3789e-01, -2.6232e-01,  1.6171e-03,  3.0519e-01,
         -6.0726e-01,  4.0246e-01, -1.5949e-01, -2.9613e-01, -3.1445e-02,
          1.3803e-01,  9.1508e-02,  2.2612e-01, -8.3417e-02,  2.1938e-01]],
       grad_fn=<ViewBackward>)

In [103]:
test_lstm_new_out2

tensor([[-0.7324, -0.0926,  0.1145,  0.2461, -0.1540, -0.2603,  0.2289,  0.0330,
         -0.2470,  0.0690,  0.0489, -0.1343, -0.3406, -0.2146,  0.5267, -0.0347,
          0.0228,  0.5530,  0.0456,  0.1882],
        [ 0.0426,  0.2714,  0.0720, -0.1700, -0.3775, -0.3128, -0.1702, -0.2518,
          0.0155,  0.2878, -0.5411,  0.4220, -0.1490, -0.2301, -0.0282,  0.1775,
          0.0708,  0.2244, -0.1130,  0.2346]], grad_fn=<ViewBackward>)

In [133]:
test_lstm_new_out = torch.cat((test_lstm_new_out1, test_lstm_new_out2), dim = 0).flatten()
test_lstm_new_out = test_lstm_new_out.view(1, test_lstm_new_out.size(0))
test_lstm_new_out

tensor([[-5.1465e-01, -1.6745e-01,  3.1715e-02,  1.8416e-01, -1.7783e-01,
         -2.7314e-01,  2.8250e-01,  4.5750e-02, -2.4371e-01, -2.4981e-02,
          3.5792e-02, -1.4749e-01, -3.1761e-01, -2.4772e-01,  4.9829e-01,
         -4.7374e-02, -5.3364e-04,  4.8119e-01,  3.1403e-02,  1.5268e-01,
          1.4047e-02,  3.5429e-01, -3.0918e-02, -1.1251e-01, -3.2316e-01,
         -3.1126e-01, -1.3789e-01, -2.6232e-01,  1.6171e-03,  3.0519e-01,
         -6.0726e-01,  4.0246e-01, -1.5949e-01, -2.9613e-01, -3.1445e-02,
          1.3803e-01,  9.1508e-02,  2.2612e-01, -8.3417e-02,  2.1938e-01,
         -7.3243e-01, -9.2614e-02,  1.1453e-01,  2.4609e-01, -1.5401e-01,
         -2.6027e-01,  2.2886e-01,  3.2998e-02, -2.4700e-01,  6.8988e-02,
          4.8853e-02, -1.3435e-01, -3.4062e-01, -2.1463e-01,  5.2670e-01,
         -3.4679e-02,  2.2761e-02,  5.5298e-01,  4.5591e-02,  1.8819e-01,
          4.2639e-02,  2.7135e-01,  7.1964e-02, -1.7001e-01, -3.7750e-01,
         -3.1278e-01, -1.7021e-01, -2.

In [134]:
test_hiddenlinear = nn.Linear(4*hidden_dim2, hidden_dim3)
test_predlinear = nn.Linear(hidden_dim3, 2)

In [135]:
hidden_result = test_hiddenlinear(test_lstm_new_out)
hidden_result

tensor([[-1.4285e-01,  7.5127e-02,  1.8495e-01,  4.7088e-02, -2.4231e-01,
         -1.2013e-01,  1.2017e-01,  1.4376e-01, -2.6500e-01, -1.1666e-01,
          6.5637e-02,  2.4466e-01, -1.5559e-01,  6.1745e-02, -6.7375e-02,
         -2.2390e-02,  1.3145e-01,  2.2541e-01, -3.6408e-02,  1.7200e-01,
          1.2756e-01, -1.4357e-01, -2.1728e-02,  1.3567e-01, -8.1019e-03,
          2.1885e-01,  1.8540e-01, -2.0861e-02, -1.9221e-01,  3.4208e-01,
          3.3577e-02,  7.2601e-03,  1.2733e-01, -8.4238e-02, -2.8682e-01,
         -1.1801e-01,  1.8170e-01, -7.1025e-02, -6.6506e-04,  6.4207e-02,
          3.4252e-02, -2.9310e-04, -1.3552e-02,  7.4504e-02,  2.9108e-01,
         -1.4040e-01,  1.8030e-02, -1.8633e-01,  1.4907e-01,  4.6020e-02,
         -1.3210e-01,  3.2084e-01,  1.4025e-01, -4.5091e-02,  2.0774e-01,
         -2.2954e-01, -3.4594e-02, -2.0626e-02,  4.0186e-01, -2.6521e-02,
          1.9059e-01, -3.8375e-01,  2.5545e-03, -2.3291e-01,  2.6083e-01,
          2.2062e-01, -3.4279e-02, -1.

In [136]:
final_result = test_predlinear(hidden_result)
final_result

tensor([[ 0.0665, -0.1767]], grad_fn=<AddmmBackward>)

In [143]:
tag_scores = F.softmax(final_result, dim=1)
tag_scores

tensor([[0.5605, 0.4395]], grad_fn=<SoftmaxBackward>)

### Model

In [34]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [35]:
embedding_dim = vocab_size
embedding_dim

45865

In [36]:
hidden_dim = 100
match_size = 20
hidden_dim2 = 100
hidden_dim3 = 100

In [87]:
embedding_matrix_copy = np.copy(embedding_matrix)

In [88]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, match_size, hidden_dim2, hidden_dim3):
        super(LSTMTagger, self).__init__()
        
        self.embedding = nn.Embedding(embedding_dim, 100)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix_copy, dtype=torch.float32)) 

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(input_size = 100, hidden_size = hidden_dim, batch_first = True, bidirectional = True, dropout = 0.1)
        
        # Matching layer
        self.matchpara1 = nn.Parameter(torch.rand(match_size, hidden_dim))
        self.matchpara2 = nn.Parameter(torch.rand(match_size, hidden_dim))
        self.cosSim = nn.CosineSimilarity(dim=1)
        
        # Aggregation layer
        self.lstm_new = nn.LSTM(input_size = match_size*2, hidden_size = hidden_dim2, batch_first=True, bidirectional = True, dropout = 0.1)
        
        # Prediction layer
        self.hiddenlinear = nn.Linear(4*hidden_dim2, hidden_dim3)
        self.predlinear = nn.Linear(hidden_dim3, 2)
        
    def forward(self, input1, input2):
        #print(input1.size())
        embeds1 = self.embedding(input1)
        #print(embeds1.size())
        lstm_out1, _ = self.lstm(embeds1)
        embeds2 = self.embedding(input2)
        lstm_out2, _ = self.lstm(embeds2)
        
        # matching
        match_1 = self.match_cal(lstm_out1, lstm_out2)
        match_2 = self.match_cal(lstm_out2, lstm_out1)
        
        # aggregating
        _,(lstm_new_out1,_) = self.lstm_new(match_1)
        _,(lstm_new_out2,_) = self.lstm_new(match_2)
        
        # predicting
        lstm_new_out1 = lstm_new_out1.view(lstm_new_out1.size(0), lstm_new_out1.size(2))
        lstm_new_out2 = lstm_new_out2.view(lstm_new_out2.size(0), lstm_new_out2.size(2))
        concat_out = torch.cat((lstm_new_out1, lstm_new_out2), dim = 0).flatten()
        concat_out = concat_out.view(1, concat_out.size(0))
        hidden_result = self.hiddenlinear(concat_out)
        final_result = self.predlinear(hidden_result)
        tag_scores = F.softmax(final_result, dim=1)
        
        return tag_scores
    
    def match_cal(self, lstm_out1, lstm_out2):
        match_out_all = []
        for b in range(lstm_out1.size(0)):
            match_out_lst = []
            lstm_out1_batch1 = lstm_out1[b,:,:]
            #lstm_out2_batch1_end = lstm_out2[b, -1, :]
            for seq in range(lstm_out1_batch1.size(0)):
                batch1_t_first = lstm_out1_batch1[seq,:hidden_dim]
                #batch2_t = lstm_out2_batch1_end
                batch2_t_first = self.matching_method(batch1_t_first, lstm_out2[b,:,:hidden_dim])
                mod_batch1_t_first = torch.mul(self.matchpara1, batch1_t_first)
                mod_batch2_t_first = torch.mul(self.matchpara1, batch2_t_first)
                match_out_t_first = self.cosSim(mod_batch1_t_first, mod_batch2_t_first)
                
                batch1_t_second = lstm_out1_batch1[seq,hidden_dim:]
                batch2_t_second = self.matching_method(batch1_t_second, lstm_out2[b,:,hidden_dim:])
                mod_batch1_t_second = torch.mul(self.matchpara2, batch1_t_second)
                mod_batch2_t_second = torch.mul(self.matchpara2, batch2_t_second)
                match_out_t_second = self.cosSim(mod_batch1_t_second, mod_batch2_t_second)
                
                match_out_t = torch.cat((match_out_t_first, match_out_t_second), dim=0)
                match_out_lst.append(match_out_t)
            match_out = torch.stack(match_out_lst)
            match_out_all.append(match_out)
        return torch.stack(match_out_all)
    
    def matching_method(self, batch1_t, lstm_out2_batch1):
        prep_batch1_t = batch1_t.repeat(lstm_out2_batch1.size(0),1)
        weight = self.cosSim(prep_batch1_t, lstm_out2_batch1).detach().numpy()
        max_weight = np.argmax(weight)
        return lstm_out2_batch1[max_weight,:]

### Train and evaluation

In [38]:
Batch_size = 32

In [39]:
from sklearn.metrics import f1_score

In [89]:
LSTMmodel = LSTMTagger(embedding_dim, hidden_dim, match_size, hidden_dim2, hidden_dim3)
loss_fn = nn.BCEWithLogitsLoss(reduction='sum')
optimizer = optim.Adam(LSTMmodel.parameters(), lr=(1e-6))

In [90]:
n_epochs = 5

In [72]:
num_train_batch = np.ceil(len(X_train_1)/Batch_size)
num_train_batch

2499.0

In [73]:
num_valid_batch = np.ceil(len(X_valid_1) / Batch_size)
num_valid_batch

625.0

#### <span style="color:blue">Test</span>

In [194]:
Y_train[1]

tensor([1.])

In [195]:
Y_train[1].size()

torch.Size([1])

In [196]:
tag_scores[:,-1]

tensor([3.2177e-08])

In [197]:
tag_scores[:,-1].size()

torch.Size([1])

In [198]:
loss_fn(tag_scores[:,-1], Y_train[0])

tensor(0.6931)

In [91]:
for epoch in range(n_epochs):  
    # Train
    LSTMmodel.train()
    X_train_1, X_train_2, Y_train = shuffle(X_train_1, X_train_2, Y_train)
    
    avg_loss = 0. 
    batch_num = 1
    for n in range(len(X_train_1)):
        # Step 0. Split the batch
        
        if n % Batch_size == 0:
            # Step 1. Remember that Pytorch accumulates gradients.
            # We need to clear them out before each instance
            LSTMmodel.zero_grad()
            loss = 0
        # Step 2. Run our forward pass.
        tag_scores = LSTMmodel(X_train_1[n], X_train_2[n])

        # Step 3. Compute the loss
        loss += loss_fn(tag_scores[:,-1], Y_train[n])
        
        # Step 4. Compute gradients, and update the parameters by calling optimizer.step()
        if (n % Batch_size == Batch_size - 1) or n == len(X_train_1)-1:
            loss.backward()
            optimizer.step()
            avg_loss += loss.item() / num_train_batch
            if batch_num % 20 == 1:
                print('Batch {} \t loss={:4f}'.format(batch_num, avg_loss))
            batch_num += 1
        
    # Evaluation
    LSTMmodel.eval()
    X_valid_1, X_valid_2, Y_valid = shuffle(X_valid_1, X_valid_2, Y_valid)
    
    batch_num = 1
    avg_metric = 0.
    for n in range(len(X_valid_1)):
        if n % Batch_size == 0:
            predict_scores, real_labels = [], []
        with torch.no_grad():
            tag_scores = LSTMmodel(X_valid_1[n], X_valid_2[n])
            
        predict_label = np.argmax(tag_scores.detach().numpy(), axis=1).flatten()
        predict_scores.append(predict_label)
        real_labels.append(Y_valid[n].numpy())
        if (n % Batch_size == Batch_size - 1) or n == len(X_valid_1)-1:
            avg_metric += f1_score(predict_scores, real_labels) / num_valid_batch
            if batch_num % 20 == 1:
                print('Batch {} \t metric={:4f}'.format(batch_num, avg_metric))
            batch_num += 1
        
    # Output
    print('Epoch {}/{} \t loss={:.4f} \t metric={:.4f}'.format(
                epoch + 1, n_epochs, avg_loss, avg_metric))

Batch 1 	 loss=0.010430
Batch 21 	 loss=0.216342
Batch 41 	 loss=0.420842
Batch 61 	 loss=0.627156
Batch 81 	 loss=0.834446
Batch 101 	 loss=1.042223
Batch 121 	 loss=1.248537
Batch 141 	 loss=1.455592
Batch 161 	 loss=1.656785
Batch 181 	 loss=1.859360
Batch 201 	 loss=2.069614
Batch 221 	 loss=2.271307
Batch 241 	 loss=2.476224
Batch 261 	 loss=2.678895
Batch 281 	 loss=2.878839
Batch 301 	 loss=3.083795
Batch 321 	 loss=3.290498
Batch 341 	 loss=3.494165
Batch 361 	 loss=3.700553
Batch 381 	 loss=3.902904
Batch 401 	 loss=4.103524
Batch 421 	 loss=4.306669
Batch 441 	 loss=4.505113
Batch 461 	 loss=4.706044
Batch 481 	 loss=4.908590
Batch 501 	 loss=5.109792
Batch 521 	 loss=5.313713
Batch 541 	 loss=5.512243
Batch 561 	 loss=5.712338
Batch 581 	 loss=5.913498
Batch 601 	 loss=6.117137
Batch 621 	 loss=6.317957
Batch 641 	 loss=6.519524
Batch 661 	 loss=6.718201
Batch 681 	 loss=6.921604
Batch 701 	 loss=7.125202
Batch 721 	 loss=7.326595
Batch 741 	 loss=7.529015
Batch 761 	 loss=7

KeyboardInterrupt: 

In [92]:
import shutil
def save_ckp(state, is_best, checkpoint_dir="", best_model_dir=""):
    f_path = 'checkpoint.pt'
    torch.save(state, f_path)
    if is_best:
        best_fpath =  'best_model.pt'
        shutil.copyfile(f_path, best_fpath)

In [93]:
checkpoint = {'epoch': epoch + 1,'loss': avg_loss, 'metric': avg_metric, 'state_dict': LSTMmodel.state_dict(),'optimizer': optimizer.state_dict()}
        
# save checkpoint
save_ckp(checkpoint, True)

In [185]:
    # Evaluation
    LSTMmodel.eval()
    X_valid_1, X_valid_2, Y_valid = shuffle(X_valid_1, X_valid_2, Y_valid)
    
    batch_num = 1
    avg_metric = 0.
    for n in range(len(X_valid_1)):
        if n % Batch_size == 0:
            predict_scores, real_labels = [], []
        with torch.no_grad():
            tag_scores = LSTMmodel(X_valid_1[n], X_valid_2[n])
            
        predict_label = np.argmax(tag_scores.detach().numpy(), axis=1).flatten()
        predict_scores.append(predict_label)
        real_labels.append(Y_valid[n].numpy())
        if (n % Batch_size == Batch_size - 1) or n == len(X_valid_1)-1:
            avg_metric += f1_score(predict_scores, real_labels) / num_valid_batch
            if batch_num % 20 == 1:
                print('Batch {} \t metric={:4f}'.format(batch_num, avg_metric))
            batch_num += 1

Batch 1 	 metric=0.000000
Batch 21 	 metric=0.000000
Batch 41 	 metric=0.000000
Batch 61 	 metric=0.000000
Batch 81 	 metric=0.000000
Batch 101 	 metric=0.000000
Batch 121 	 metric=0.000000
Batch 141 	 metric=0.000000
Batch 161 	 metric=0.000000
Batch 181 	 metric=0.000000
Batch 201 	 metric=0.000000
Batch 221 	 metric=0.000000
Batch 241 	 metric=0.000000
Batch 261 	 metric=0.000000
Batch 281 	 metric=0.000000
Batch 301 	 metric=0.000000
Batch 321 	 metric=0.000000
Batch 341 	 metric=0.000000
Batch 361 	 metric=0.000000
Batch 381 	 metric=0.000000
Batch 401 	 metric=0.000000
Batch 421 	 metric=0.000000
Batch 441 	 metric=0.000000
Batch 461 	 metric=0.000000
Batch 481 	 metric=0.000000
Batch 501 	 metric=0.000000
Batch 521 	 metric=0.000000
Batch 541 	 metric=0.000000
Batch 561 	 metric=0.000000
Batch 581 	 metric=0.000000
Batch 601 	 metric=0.000000
Batch 621 	 metric=0.000000


In [53]:
tag_scores

tensor([[9.9985e-01, 1.5031e-04]])